<a href="https://colab.research.google.com/github/shantanu2383/Ball-2020/blob/main/Copy_of_0304_CROSS_SECTION_OF_EXPECTED_RETURNS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earnings, Retained Earnings and B/M in the cross section of expected returns

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)


import pandas as pd
import statsmodels.formula.api as smf 
import numpy as np 
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA 
from sklearn.metrics import mean_squared_error, mean_absolute_error 

import math
import matplotlib.pyplot as plt 
from datetime import datetime
import seaborn as sns 
import pandasql as ps

from sqlite3 import connect
conn=connect(':memory:')

Mounted at /content/gdrive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26782 sha256=42986f2d8ebc168bcb3775c12706d50a082c5b7ac60ca623cd2fd694a1a719f2
  Stored in directory: /root/.cache/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


# DATALOAD

CRSP:

In [ ]:
filepath="/content/gdrive/MyDrive/B_M/"

file="CRSPMONTHLY.csv"
crsp=pd.read_csv(filepath + file)

crsp['date'] = pd.to_datetime(crsp['date'], format="%Y%m%d")


crsp=crsp[("1960-01-01"<=crsp['date']) & (crsp['date']< "2019-01-01")]
crsp_parsed=crsp[['PERMNO', 'date', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD', 'VOL']]
crsp=crsp_parsed


x=['PERMNO', 'RET', 'SHROUT', 'ALTPRC', 'EXCHCD', 'SHRCD', 'SICCD', 'DLRET', 'DLSTCD']

for i in x:
  crsp[i]=pd.to_numeric(crsp[i], errors='coerce')


#convert returns to percent:
crsp['RET']*=100
crsp['DLRET']*=100


#add market cap column
crsp['mkt_cap']=abs(crsp['SHROUT'] * crsp['ALTPRC'])/1000
crsp['mkt_cap'].replace(0, np.NaN, inplace=True)


#ONLY KEEP US BASED COMMON STOCKS
crsp=crsp[(crsp['SHRCD']==10)|(crsp['SHRCD']==11)]

#rename variables to lower case
crsp= crsp.rename(columns=str.lower)

#filter for relevant exchanges
exchange_mapping = {
    1: 'NYSE', 31: 'NYSE',
    2: 'AMEX', 32: 'AMEX',
    3: 'NASDAQ', 33: 'NASDAQ'
}
#create dictionary to map values of the 'exchcd' column
crsp['exchange'] = np.select(
    [crsp['exchcd'].isin(exchange_mapping.keys()), crsp['exchcd'].notnull()],
    [crsp['exchcd'].map(exchange_mapping), "Other"],
    default='Other')


#Adjust Returns
crsp['ret_adj'] = crsp['ret']

mask1 = pd.isnull(crsp['dlstcd'])
crsp.loc[mask1, 'ret_adj'] = crsp.loc[mask1, 'ret']

mask2 = pd.notnull(crsp['dlstcd']) & pd.notnull(crsp['dlret'])
crsp.loc[mask2, 'ret_adj'] = crsp.loc[mask2, 'dlret']

mask3 = ((551 <= crsp['dlstcd']) & (crsp['dlstcd'] <= 574)) | (crsp['dlstcd'].isin([500, 520, 580, 584]))
crsp.loc[mask3, 'ret_adj'] = -30

crsp.loc[~(mask1 | mask2 | mask3), 'ret_adj'] = -100

crsp.drop(['shrcd', 'dlret', 'dlstcd'], inplace=True, axis=1)

#exclude financial firms 
crsp=crsp[(crsp['siccd']<6000) |(crsp['siccd']>6999) ]


#Add reference date for matching with Compustat

crsp['date']=pd.to_datetime(crsp['date'])
crsp['year']=crsp['date'].dt.year

crsp['reference_date']=pd.to_datetime(crsp['year'].astype(str) +'-06-01')

crsp.loc[crsp['date'].dt.month < 6, 'year'] -= 1

crsp.loc[crsp['date'].dt.month < 6, 'reference_date'] = pd.to_datetime(
    crsp['year'].astype(str) + '-06-01')

crsp = crsp.drop('year', axis=1)

crsp.set_index('date', inplace=True)

crsp.sort_values(by=['date', 'permno'], inplace=True)

<ipython-input-2-0ead0e28d922>:4: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp=pd.read_csv(filepath + file)
<ipython-input-2-0ead0e28d922>:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp['date']=pd.to_datetime(crsp['date'])


In [ ]:
#drop any securities that do not have observations for t-12 returns
columns=[]
for x in range(1,13):
  label=f"return_t-{x}"
  columns.append(label)
  crsp[f'return_t-{x}']=crsp.groupby('permno')['ret_adj'].shift(x)

crsp.dropna(subset=columns, inplace=True)

crsp.dropna(subset=['ret_adj'], inplace=True)

Compustat:

In [ ]:
filepath="/content/gdrive/MyDrive/B_M/"

compustat=pd.read_csv(filepath + "compustat.csv")
compustat

#parse relevant variables from compustat
compustat= compustat.rename(columns=str.lower)
x=['gvkey', 'lpermno', 'datadate', 'linktype', 'linkenddt', 'seq', 'ceq',
                    'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstkl', 'pstk', 'indfmt', 'datafmt',
                     'revt', 'cogs', 'xint', 'xsga', 'xrd', 'rect', 'invt', 'xpp', 'drc', 'drlt', 'ap', 'xacc',
                      'cstk', 'caps', 'tstk', 're', 'acominc', 'mkvalt']

compustat=compustat[x]


dates=['datadate', 'linkenddt']
for date in dates:
  compustat[date]=pd.to_datetime(compustat[date], format="%Y%m%d", errors='coerce')

#filter columns
compustat['datafmt']=compustat['datafmt'].apply(str)
compustat=compustat[compustat["indfmt"]=="INDL"]
compustat=compustat[compustat['datafmt']=="STD"]

#Only keep valid links
compustat=compustat[(compustat["linktype"]=="LU")| (compustat["linktype"]=="LC")]

#Only keep links active at datadate
compustat=compustat[(compustat['datadate']<=compustat["linkenddt"]) | pd.isnull(compustat['linkenddt'])]

#calculate book value using FF definition
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT *,
        COALESCE(seq, ceq+pstk, at-lt) + COALESCE(txditc, txdb + itcb, 0)- COALESCE(pstkrv, pstkl, pstk, 0) as book_value
        FROM compustat'''

compustat_calc=pd.read_sql(query, conn)

compustat=compustat_calc
#add reference date for matching- We lag accounting information by 6 months,
#therefore if a firms financial year ends in June, we assume this information is
#available to investors the following June
compustat['datadate']=pd.to_datetime(compustat['datadate'], errors='coerce')

compustat['year']=compustat['datadate'].dt.year
compustat['year']=pd.to_numeric(compustat['year'], errors='coerce')
compustat['reference_date']=compustat['year']+1
compustat['reference_date']=compustat['reference_date'].apply(str)
compustat['reference_date']+="-06-01"
compustat['reference_date']=pd.to_datetime(compustat['reference_date'])

<ipython-input-4-46f956b25077>:3: DtypeWarning: Columns (2,15,17,18,19,23,24,29,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  compustat=pd.read_csv(filepath + "compustat.csv")


In [ ]:
compustat['re'].isnull().sum()


29503

Match Compustat to Crsp on Reference Date (lagged 6 months)

In [ ]:
compustat.rename(columns={'lpermno':'permno'}, inplace=True)
#compustat.drop('index', axis=1, inplace=True)

#compustat.rename(columns={'da_a/t-1': 'da_a_t_1'}, inplace=True)
crsp.to_sql("crsp", conn, if_exists='replace')
compustat.to_sql("compustat", conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, c.gvkey, c.seq, c.ceq, c.at, c.lt, 
        c.txditc, c.txdb, c.itcb, c.pstkrv, c.pstkl, c.pstk,
        c.revt, c.cogs, c.xint, c.xsga, c.xrd, c.rect, c.invt,
        c.xpp, c.drc, c.drlt, c.ap, c.xacc, c.cstk, c.caps, c.tstk,
        c.re, c.acominc, c.mkvalt, c.book_value
        FROM crsp
        LEFT JOIN compustat c
        ON crsp.permno=c.permno
        AND crsp.reference_date=c.reference_date
        '''
crsp_comp=pd.read_sql(query, conn)
crsp=crsp_comp


Extract market cap at end of year as 'market value of equity'

In [ ]:
crsp['date'] = pd.to_datetime(crsp['date'])
me=crsp[(crsp['date'].dt.month==12)]
me['reference_date']=(me['date'].dt.year) + 1
me['reference_date']=me['reference_date'].apply(str)
me['reference_date']+="-06-01"
me['reference_date']=pd.to_datetime(me['reference_date'])

me.to_sql('me', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, mkt_cap as market_equity
        FROM me
        '''
me=pd.read_sql(query, conn)


crsp.to_sql('crsp', conn, if_exists='replace')
me.to_sql('me', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, me.market_equity
        FROM crsp
        LEFT JOIN me
        ON crsp.permno=me.permno
        AND crsp.reference_date=me.reference_date
        '''
crsp_me=pd.read_sql(query, conn)

crsp=crsp_me

<ipython-input-7-acd63386657f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me['reference_date']=(me['date'].dt.year) + 1
<ipython-input-7-acd63386657f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  me['reference_date']=me['reference_date'].apply(str)
<ipython-input-7-acd63386657f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

In [ ]:
crsp.to_csv(filepath+ "crsp_compustat_clean.csv")

Filter the Sample Accordingly

In [ ]:
filepath="/content/gdrive/MyDrive/B_M/"

crsp=pd.read_csv(filepath +"crsp_compustat_clean.csv" )

In [ ]:
#Filter 2: Non-Missing Values for book value and market equity
# Drop missing values of market equity
crsp.dropna(subset=['market_equity'], inplace=True)

# Drop rows where both ceq and book_value are missing
crsp.dropna(subset=['ceq', 'book_value'], how='all', inplace=True)

#Filter 1: Non-Missing Values for current month returns and returns for the prior
#1 year period
'''
columns=[]
for x in range(1,13):
  label=f"return_t-{x}"
  columns.append(label)
  crsp[f'return_t-{x}']=crsp.groupby('permno')['ret_adj'].shift(x)

crsp.dropna(subset=columns, inplace=True)

crsp.dropna(subset=['ret_adj'], inplace=True)

#Filter 2: Non-Missing Values for book value and market equity
crsp.dropna(subset=['book_value', 'market_equity', 'ceq'], inplace=True)
'''



'\ncolumns=[]\nfor x in range(1,13):\n  label=f"return_t-{x}"\n  columns.append(label)\n  crsp[f\'return_t-{x}\']=crsp.groupby(\'permno\')[\'ret_adj\'].shift(x)\n\ncrsp.dropna(subset=columns, inplace=True)\n\ncrsp.dropna(subset=[\'ret_adj\'], inplace=True)\n\n#Filter 2: Non-Missing Values for book value and market equity\ncrsp.dropna(subset=[\'book_value\', \'market_equity\', \'ceq\'], inplace=True)\n'

In [ ]:
#construct components of book value

#CEQ=Contributed Capital + Retained Earnings + Other

#Contributed Capital= CSTK + CAPS + TSTK
crsp['tstk'].fillna(0, inplace=True)
crsp['cc']=crsp['cstk']+crsp['caps']-crsp['tstk']
#Retained Earnings= RE
#Other=ACOMINC
crsp['cc'] = crsp['cc'].clip(lower=-15000)

#Construct Book Value Componenents Scaled by Market Equity
crsp['book_to_market']=crsp['book_value']/crsp['market_equity']
crsp['reported_book_to_market']=crsp['ceq']/crsp['market_equity']
crsp['contributed_capital_to_market']=crsp['cc']/crsp['market_equity']
crsp['retained_earnings_to_market']=crsp['re']/crsp['market_equity']
crsp['other_book_to_market']=crsp['acominc']/crsp['market_equity']

#Construct Book Value Components Scaled by Book Value of Equity
crsp['re/b']=crsp['re']/crsp['ceq']
crsp['cc/b']=crsp['cc']/crsp['ceq']
crsp['other/b']=crsp['acominc']/crsp['ceq']

In [ ]:
crsp.to_csv(filepath+ "crsp_bv.csv")

#Table 1 Panel A

In [ ]:
crsp=crsp[("1964-07-01"<=crsp['date']) & (crsp['date']< "2018-01-01")]

In [ ]:

#Construct dataframe
df = pd.DataFrame(columns=['Ratio', 'Mean', 'std_dev', 'Pct < 0', '10th', '25th', '50th', '75th', '90th'])

for x, label in zip(['book_to_market', 'reported_book_to_market',  'retained_earnings_to_market','contributed_capital_to_market', 'other_book_to_market'], ['Book-to-market', 'Reported book-to-market', 'Retained earnings-to-market', 'Contributed capital-to-market', 'Other book-to-market']):
  mean=crsp[x].mean()
  std_dev=crsp[x].std()

  #calculate percentage of values <0
  num_neg_values = (crsp[x] < 0).sum()
  percent_neg_values = num_neg_values /len(crsp) * 100
  
  quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
  results = crsp[x].quantile(quantiles)

  new_row = {'Ratio': label, 'Mean': mean, 'std_dev':std_dev, 'Pct < 0': percent_neg_values, '10th': results.loc[0.1], '25th':results.loc[0.25], '50th': results.loc[0.5], '75th': results.loc[0.75], '90th': results.loc[0.9]}
  df = df.append(new_row, ignore_index=True)

df=df.round(2)
df


<ipython-input-14-f020ab32978c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-14-f020ab32978c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-14-f020ab32978c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-14-f020ab32978c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-14-f020ab32978c>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Us

,Ratio,Mean,std_dev,Pct < 0,10th,25th,50th,75th,90th
0,Book-to-market,0.83,3.64,2.96,0.15,0.33,0.63,1.10,1.79
1,Reported book-to-market,0.80,3.31,2.57,0.15,0.32,0.60,1.04,1.71
2,Retained earnings-to-market,-0.13,5.09,28.04,-0.89,-0.06,0.21,0.52,0.98
3,Contributed capital-to-market,0.93,4.20,4.39,0.04,0.14,0.36,0.81,1.82
4,Other book-to-market,-0.02,0.24,13.69,-0.05,-0.01,0.00,0.00,0.01


Now we edit the sample to exclude firms where book value of equity is negative, and any of the componenents are not between -100% and 200% of the book value of equity.

In [ ]:
crsp=crsp[crsp['book_value']>0]
# Define the lower and upper bounds
ceq = crsp['ceq']
lower_lim = -1 * ceq
upper_lim = 2* ceq

# Filter the DataFrame
crsp_filtered = crsp[((crsp['cc'].isna()) | ((crsp['cc'] >= lower_lim) & (crsp['cc'] <= upper_lim))) &
                     ((crsp['re'].isna()) | ((crsp['re'] >= lower_lim) & (crsp['re'] <= upper_lim))) &
                     ((crsp['acominc'].isna()) | ((crsp['acominc'] >= lower_lim) & (crsp['acominc'] <= upper_lim)))]

crsp=crsp_filtered


In [ ]:
df = pd.DataFrame(columns=['Ratio', 'Mean', 'std_dev', 'Pct < 0', '10th', '25th', '50th', '75th', '90th'])

for x, label in zip(['re/b', 'cc/b',  'other/b'], ['Retained Earnings(R/E)', 'Contributed Capital (CC)', 'Other(AOCI)']):
  mean=crsp[x].mean()
  std_dev=crsp[x].std()

  #calculate percentage of values <0
  num_neg_values = (crsp[x] < 0).sum()
  percent_neg_values = num_neg_values /len(crsp) *100
  
  quantiles = [0.1, 0.25, 0.5, 0.75, 0.9]
  results = crsp[x].quantile(quantiles)

  new_row = {'Ratio': label, 'Mean': mean, 'std_dev':std_dev, 'Pct < 0': percent_neg_values, '10th': results.loc[0.1], '25th':results.loc[0.25], '50th': results.loc[0.5], '75th': results.loc[0.75], '90th': results.loc[0.9]}
  df = df.append(new_row, ignore_index=True)

df=df.round(2)
df

<ipython-input-16-570d679462d8>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-16-570d679462d8>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-16-570d679462d8>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


,Ratio,Mean,std_dev,Pct < 0,10th,25th,50th,75th,90th
0,Retained Earnings(R/E),0.45,0.46,14.81,-0.19,0.21,0.52,0.77,0.92
1,Contributed Capital (CC),0.55,0.46,4.32,0.08,0.23,0.48,0.79,1.19
2,Other(AOCI),-0.02,0.10,12.40,-0.09,-0.02,-0.00,0.00,0.02


# Table 1 Panel B

In [ ]:
crsp=pd.read_csv(filepath + "crsp_bv.csv" )
crsp['date']=pd.to_datetime(crsp['date'])
crsp=crsp[crsp['book_value']>0]
# Define the lower and upper bounds
ceq = crsp['ceq']
lower_lim = -1 * ceq
upper_lim = 2* ceq

# Filter the DataFrame
crsp_filtered = crsp[((crsp['cc'].isna()) | ((crsp['cc'] >= lower_lim) & (crsp['cc'] <= upper_lim))) &
                     ((crsp['re'].isna()) | ((crsp['re'] >= lower_lim) & (crsp['re'] <= upper_lim))) &
                     ((crsp['acominc'].isna()) | ((crsp['acominc'] >= lower_lim) & (crsp['acominc'] <= upper_lim)))]

crsp=crsp_filtered


In [ ]:
crsp

Fama French Size Sort

In [ ]:
crsp['date']=pd.to_datetime(crsp['date'])
sb=crsp[(crsp['date'].dt.month==6) & (crsp['exchange']=="NYSE")]
sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, mkt_cap as ff_mkt_equity
        FROM sb
        '''

sb=pd.read_sql(query, conn)

#get median market equity for nyse
sb['median_mkt_cap'] = sb.groupby('reference_date')['ff_mkt_equity'].transform('median')

sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, median_mkt_cap
        FROM SB
        '''
sb=pd.read_sql(query, conn)

crsp.to_sql('crsp', conn, if_exists='replace')
sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, sb.median_mkt_cap
        FROM crsp
        LEFT JOIN sb
        ON crsp.reference_date=sb.reference_date
        '''
crsp_sb=pd.read_sql(query, conn)
crsp=crsp_sb
crsp.dropna(subset=['median_mkt_cap'], inplace=True)

crsp['size_portfolio']=np.where(crsp['market_equity']>crsp['median_mkt_cap'], "B", "S")

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
crsp

Fama French B/M Sort

In [ ]:
crsp['date']=pd.to_datetime(crsp['date'])
bb=crsp[crsp['exchange']=="NYSE"]
bb.drop('level_0', axis=1, inplace=True)
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, book_to_market
        FROM bb
        '''

bb=pd.read_sql(query, conn)


bb['bv_30'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.3))
bb['bv_70'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.7))

bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, bv_30, bv_70
        FROM bb
        '''
bb=pd.read_sql(query, conn)

crsp.drop(['level_0', 'Unnamed: 0', 'index'], axis=1, inplace=True)
crsp.to_sql('crsp', conn, if_exists='replace')
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, bb.bv_30, bb.bv_70
        FROM crsp
        LEFT JOIN bb
        ON crsp.reference_date=bb.reference_date
        '''
crsp_bb=pd.read_sql(query, conn)

crsp=crsp_bb

crsp['b/m_portfolio'] = np.where(crsp['book_to_market'] < crsp['bv_30'], 'growth', 
                               np.where(crsp['book_to_market'] > crsp['bv_70'], 'value', 'neutral'))


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [ ]:
crsp=crsp[("1964-07-01"<=crsp['date']) & (crsp['date']< "2018-01-01")]

In [ ]:
df = pd.DataFrame(columns=['Size', 'B/M', 'Mean', 'Median', 'PCT<0'])

for x in ["S", "B"]:
  for j in ['growth', 'neutral', 'value']:
    crsp_temp=crsp[(crsp['size_portfolio']==x) & (crsp['b/m_portfolio']==j)]
    mean=crsp_temp['re/b'].mean()
    median=crsp_temp['re/b'].median()

    #calculate percentage of values <0
    num_neg_values = (crsp_temp['re/b'] < 0).sum()
    percent_neg_values = num_neg_values /len(crsp_temp) * 100

    new_row = {'Size': x, 'B/M': j, 'Mean': mean, 'Median': median, 'PCT<0':percent_neg_values}
    df = df.append(new_row, ignore_index=True)

re_be=df.round(2)
re_be

In [ ]:
df = pd.DataFrame(columns=['Size', 'B/M', 'Mean', 'Median', 'PCT<0'])

for x in ["S", "B"]:
  for j in ['growth', 'neutral', 'value']:
    crsp_temp=crsp[(crsp['size_portfolio']==x) & (crsp['b/m_portfolio']==j)]
    mean=crsp_temp['cc/b'].mean()
    median=crsp_temp['cc/b'].median()

    #calculate percentage of values <0
    num_neg_values = (crsp_temp['cc/b'] < 0).sum()
    percent_neg_values = num_neg_values /len(crsp_temp) * 100

    new_row = {'Size': x, 'B/M': j, 'Mean': mean, 'Median': median, 'PCT<0':percent_neg_values}
    df = df.append(new_row, ignore_index=True)
cc_be=df.round(2)

In [ ]:
df = pd.DataFrame(columns=['Size', 'B/M', 'Mean', 'Median', 'PCT<0'])

for x in ["S", "B"]:
  for j in ['growth', 'neutral', 'value']:
    crsp_temp=crsp[(crsp['size_portfolio']==x) & (crsp['b/m_portfolio']==j)]
    mean=crsp_temp['other/b'].mean()
    median=crsp_temp['other/b'].median()

    #calculate percentage of values <0
    num_neg_values = (crsp_temp['other/b'] < 0).sum()
    percent_neg_values = num_neg_values /len(crsp_temp) * 100

    new_row = {'Size': x, 'B/M': j, 'Mean': mean, 'Median': median, 'PCT<0':percent_neg_values}
    df = df.append(new_row, ignore_index=True)
aoci_be=df.round(2)
aoci_be

,Size,B/M,Mean,Median,PCT<0
0,S,growth,-0.01,0.00,9.03
1,S,neutral,-0.02,0.00,11.77
2,S,value,-0.01,0.00,10.52
3,B,growth,-0.05,-0.00,18.78
4,B,neutral,-0.04,-0.01,17.33
5,B,value,-0.02,-0.01,18.74


# Table 2 Fama McBeth Regressions

First exclude microcaps from the sample, securities below the NYSE 20% Market Cap Quantile.

In [ ]:
filepath="/content/gdrive/MyDrive/B_M/"
crsp=pd.read_csv(filepath + "crsp_bv.csv" )

crsp['date']=pd.to_datetime(crsp['date'])
sb=crsp[(crsp['date'].dt.month==6) & (crsp['exchange']=="NYSE")]
sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, mkt_cap as ff_mkt_equity
        FROM sb
        '''

sb=pd.read_sql(query, conn)

#get 20% quantile of market equity for NYSE
sb['microcap_mkt_cap'] = sb.groupby('reference_date')['ff_mkt_equity'].transform(lambda x: x.quantile(0.2))

sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT reference_date, microcap_mkt_cap
        FROM SB
        '''
sb=pd.read_sql(query, conn)

crsp.to_sql('crsp', conn, if_exists='replace')
sb.to_sql('sb', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, sb.microcap_mkt_cap
        FROM crsp
        LEFT JOIN sb
        ON crsp.reference_date=sb.reference_date
        '''
crsp_sb=pd.read_sql(query, conn)
crsp=crsp_sb


#Drop microcaps
crsp = crsp[crsp['market_equity'] >= crsp['microcap_mkt_cap']]


/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:2872: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


KeyboardInterrupt: ignored

In [ ]:
filepath="/content/gdrive/MyDrive/B_M/"
crsp=pd.read_csv(filepath + "crsp_bv.csv" )


In [ ]:
crsp['ret_adj']

0           6.6770
1          -7.8571
2          -1.3274
3          -7.5342
4          -4.7619
            ...   
2030839   -29.8879
2030840    -9.3383
2030841   -10.9048
2030842   -34.1818
2030843    -5.0445
Name: ret_adj, Length: 2030844, dtype: float64

Generate prior year return skipping one month

In [ ]:

for x in range(2,13):
  crsp[f'return_t-{x}']/=100
  crsp[f'return_t-{x}']+=1

crsp['return_t-2_t-12'] = np.cumprod(crsp.loc[:, 'return_t-2':'return_t-12'], axis=1).iloc[:, -1]

# Subtract 1 and convert the result back to percentage returns
crsp['return_t-2_t-12'] = (crsp['return_t-2_t-12'] - 1) * 100

Sample Period: 1964-2017

In [ ]:
#crsp.drop (['level_0', 'Unnamed: 0', 'Unnamed: 0.1'	], axis=1, inplace=True)
#crsp.drop(['level_0', 'Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
crsp['date']=pd.to_datetime(crsp['date'])
crsp=crsp[("1964-07-01"<=crsp['date']) & (crsp['date']< "2018-01-01")]

In [ ]:
crsp['tstk'] = crsp['tstk'].fillna(0)
crsp['cc']=crsp['cstk']+crsp['caps']-crsp['tstk']

Trim variables at 1% and 99%

In [ ]:
'''
for x in ['market_equity', 'book_to_market', 'reported_book_to_market', 'return_t-1', 'return_t-2_t-12']:
    # Calculate the 1st and 99th percentiles
    p1 = crsp[x].quantile(0.01)
    p99 = crsp[x].quantile(0.99)

    # Drop observations outside of the 1st and 99th percentiles
    crsp = crsp[(crsp[x] >= p1) & (crsp[x] <= p99)]
'''

"\nfor x in ['market_equity', 'book_to_market', 'reported_book_to_market', 'return_t-1', 'return_t-2_t-12']:\n    # Calculate the 1st and 99th percentiles\n    p1 = crsp[x].quantile(0.01)\n    p99 = crsp[x].quantile(0.99)\n\n    # Drop observations outside of the 1st and 99th percentiles\n    crsp = crsp[(crsp[x] >= p1) & (crsp[x] <= p99)]\n"

In [ ]:
crsp_c=crsp.copy()

In [ ]:
#positive book value of equity
#crsp = crsp[crsp['book_value'] >= 0]

#non-missing re and cc
crsp = crsp.dropna(subset=['re', 'cc'])

#Construct Book Value Componenents Scaled by Market Equity
crsp['book_to_market']=crsp['book_value']/crsp['market_equity']
crsp['reported_book_to_market']=crsp['ceq']/crsp['market_equity']
crsp['contributed_capital_to_market']=crsp['cc']/crsp['market_equity']
crsp['retained_earnings_to_market']=crsp['re']/crsp['market_equity']
crsp['other_book_to_market']=crsp['acominc']/crsp['market_equity']

#Construct Book Value Components Scaled by Book Value of Equity
crsp['re/b']=crsp['re']/crsp['ceq']
crsp['cc/b']=crsp['cc']/crsp['ceq']
crsp['other/b']=crsp['acominc']/crsp['ceq']

#create indicator variables for when components of bv are <0:
for x in ['contributed_capital_to_market', 'retained_earnings_to_market', 'other_book_to_market']:
    # Replace negative values with 0
    #crsp[x] = crsp[x].clip(lower=0)
    
    # Create an indicator column which is 1 when the values are negative
    crsp[f'{x}_neg'] = (crsp[x] < 0).astype(int)


    # Apply the function to create a new column 'new_col'


#create log of variables 
for x in ['market_equity', 'book_to_market', 'reported_book_to_market', 'contributed_capital_to_market', 'retained_earnings_to_market', 'other_book_to_market']:
    # Create a new log variable
    crsp[x+'_log'] = np.log(crsp[x])
    crsp[x+'_log'] = crsp[x+'_log'].replace([-np.inf, np.inf, np.nan], 0)

crsp['ret_adj']/=100
crsp['return_t-1']/=100
#crsp['return_t-2_t-12']/=100

/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.9/dist-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc

In [ ]:
break

SyntaxError: ignored

# 2A

In [ ]:
#build regression dataframe

a = pd.DataFrame({'regressor': ['log(me)', 'R1', 'R12', 'log(be/me)', 'log(re/me)', 'log(cc/me)', 'log(aoci/me)', 'RE<0', 'CC<0', 'AOCI<0', 'Average ADJ R2']})
a

In [ ]:
#build regression dataframe

a = pd.DataFrame({'regressor': ['log(me)', 'R1', 'R12', 'log(be/me)', 'log(re/me)', 'log(cc/me)', 'log(aoci/me)', 'RE<0', 'CC<0', 'AOCI<0', 'Average ADJ R2']})
a
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'book_to_market_log'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(1)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(1)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(1)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(be/me)', '(1)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'Average ADJ R2', '(1)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12',  'retained_earnings_to_market_log', 'retained_earnings_to_market_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'RE':params[4]*100, 'I':params[5]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(2)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(2)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(2)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(re/me)', '(2)'] = "{:.2f} ({:.2f})".format(df['RE'].mean(), df['RE'].mean()/(df['RE'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'RE<0', '(2)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(2)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'book_to_market_log', 'retained_earnings_to_market_log', 'retained_earnings_to_market_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'RE':params[5]*100, 'I':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(3)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(3)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(3)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(be/me)', '(3)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(re/me)', '(3)'] = "{:.2f} ({:.2f})".format(df['RE'].mean(), df['RE'].mean()/(df['RE'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'RE<0', '(3)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(3)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'contributed_capital_to_market_log', 'contributed_capital_to_market_neg']]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'CC':params[4]*100, 'i':params[5]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(4)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(4)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(4)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(cc/me)', '(4)'] = "{:.2f} ({:.2f})".format(df['CC'].mean(), df['CC'].mean()/(df['CC'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'CC<0', '(4)'] = "{:.2f} ({:.2f})".format(df['i'].mean(), df['i'].mean()/(df['i'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(4)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'contributed_capital_to_market_log', 'contributed_capital_to_market_neg', 'book_to_market_log']]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'CC':params[4]*100, 'i':params[5]*100, 'BM':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(5)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(5)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(5)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(cc/me)', '(5)'] = "{:.2f} ({:.2f})".format(df['CC'].mean(), df['CC'].mean()/(df['CC'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'CC<0', '(5)'] = "{:.2f} ({:.2f})".format(df['i'].mean(), df['i'].mean()/(df['i'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(be/me)', '(5)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(5)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'other_book_to_market_log', 'other_book_to_market_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'O':params[4]*100, 'I':params[5]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(6)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(6)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(6)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(aoci/me)', '(6)'] = "{:.2f} ({:.2f})".format(df['O'].mean(), df['O'].mean()/(df['O'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'AOCI<0', '(6)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(6)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'other_book_to_market_log', 'other_book_to_market_neg', 'book_to_market_log'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'O':params[4]*100, 'I':params[5]*100, 'BM':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(7)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(7)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(7)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(aoci/me)', '(7)'] = "{:.2f} ({:.2f})".format(df['O'].mean(), df['O'].mean()/(df['O'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(be/me)', '(7)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'AOCI<0', '(7)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(7)'] = df['r2'].mean()


In [ ]:
 #build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'other_book_to_market_log', 'other_book_to_market_neg', 'retained_earnings_to_market_log', 'retained_earnings_to_market_neg', 'contributed_capital_to_market_log', 'contributed_capital_to_market_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'O':params[4]*100, 'I':params[5]*100, 'RE':params[6]*100, 'RI':params[7]*100, 'CC':params[8]*100, 'CI':params[9], 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(8)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(8)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(8)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(aoci/me)', '(8)'] = "{:.2f} ({:.2f})".format(df['O'].mean(), df['O'].mean()/(df['O'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(re/me)', '(8)'] = "{:.2f} ({:.2f})".format(df['RE'].mean(), df['RE'].mean()/(df['RE'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'RE<0', '(8)'] = "{:.2f} ({:.2f})".format(df['RI'].mean(), df['RI'].mean()/(df['RI'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'loc(cc/me)', '(8)'] = "{:.2f} ({:.2f})".format(df['CC'].mean(), df['CC'].mean()/(df['CC'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'CC<0', '(8)'] = "{:.2f} ({:.2f})".format(df['CI'].mean(), df['CI'].mean()/(df['CI'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'AOCI<0', '(8)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(8)'] = df['r2'].mean()


In [ ]:
a.fillna('-', inplace=True)
a.set_index('regressor', inplace=True)
table2_a=a
table2_a

# 2B

In [ ]:
#build regression dataframe

a = pd.DataFrame({'regressor': ['log(me)', 'R1', 'R12', 'log(Reported BE/ME)', 'log(re/me)', 'log(cc/me)', 'log(aoci/me)', 'RE<0', 'CC<0', 'AOCI<0', 'Average ADJ R2']})
a
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'reported_book_to_market_log'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(1)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(1)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(1)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(Reported BE/ME)', '(1)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'Average ADJ R2', '(1)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'reported_book_to_market_log', 'retained_earnings_to_market_log', 'retained_earnings_to_market_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'RE':params[5]*100, 'I':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(2)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(2)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(2)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(Reported BE/ME)', '(2)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(re/me)', '(2)'] = "{:.2f} ({:.2f})".format(df['RE'].mean(), df['RE'].mean()/(df['RE'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'RE<0', '(2)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(2)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'contributed_capital_to_market_log', 'contributed_capital_to_market_neg', 'reported_book_to_market_log']]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'CC':params[4]*100, 'i':params[5]*100, 'BM':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(5)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(5)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(5)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(cc/me)', '(5)'] = "{:.2f} ({:.2f})".format(df['CC'].mean(), df['CC'].mean()/(df['CC'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'CC<0', '(5)'] = "{:.2f} ({:.2f})".format(df['i'].mean(), df['i'].mean()/(df['i'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(Reported BE/ME)', '(5)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(5)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'other_book_to_market_log', 'other_book_to_market_neg', 'reported_book_to_market_log'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'O':params[4]*100, 'I':params[5]*100, 'BM':params[6]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(4)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(4)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(4)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(aoci/me)', '(4)'] = "{:.2f} ({:.2f})".format(df['O'].mean(), df['O'].mean()/(df['O'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(Reported BE/ME)', '(4)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'AOCI<0', '(4)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(4)'] = df['r2'].mean()


In [ ]:
a.fillna('-', inplace=True)
a.set_index('regressor', inplace=True)
table2_b=a
table2_b

# 2C

In [ ]:
crsp=crsp_c
for x in ['contributed_capital_to_market', 'retained_earnings_to_market', 'other_book_to_market', 'book_value']:
    # Replace negative values with 0
    #crsp[x] = crsp[x].clip(lower=0)
    
    # Create an indicator column which is 1 when the values are negative
    crsp[f'{x}_neg'] = (crsp[x] < 0).astype(int)

#non-missing re and cc
crsp = crsp.dropna(subset=['re', 'cc'])

#Construct Book Value Componenents Scaled by Market Equity
crsp['book_to_market']=crsp['book_value']/crsp['market_equity']
crsp['reported_book_to_market']=crsp['ceq']/crsp['market_equity']
crsp['contributed_capital_to_market']=crsp['cc']/crsp['market_equity']
crsp['retained_earnings_to_market']=crsp['re']/crsp['market_equity']
crsp['other_book_to_market']=crsp['acominc']/crsp['market_equity']

#Construct Book Value Components Scaled by Book Value of Equity
crsp['re/b']=crsp['re']/crsp['ceq']
crsp['cc/b']=crsp['cc']/crsp['ceq']
crsp['other/b']=crsp['acominc']/crsp['ceq']



#create log of variables 
for x in ['market_equity', 'book_to_market', 'reported_book_to_market', 'contributed_capital_to_market', 'retained_earnings_to_market', 'other_book_to_market']:
    # Create a new log variable
    crsp[x+'_log'] = np.log(crsp[x])
    crsp[x+'_log'] = crsp[x+'_log'].replace([-np.inf, np.inf, np.nan], 0)


crsp['ret_adj']/=100
crsp['return_t-1']/=100
crsp['return_t-2_t-12']/=100

In [ ]:
#build regression dataframe

a = pd.DataFrame({'regressor': ['log(me)', 'R1', 'R12', 'log(BE/ME)', 'log(re/me)',  'BE<0',  'RE<0', 'Average ADJ R2']})
a
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'book_to_market_log', 'book_value_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'I':params[5]*100, 'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(1)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(1)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(1)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(BE/ME)', '(1)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'BE<0', '(1)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(1)'] = df['r2'].mean()


In [ ]:
#build dataframe to store coefficients
df = pd.DataFrame({'DATE':[], 'ME': [], 'R1': [], 'R12': [], 'cc': [], 'I': []})
dates=crsp['date'].unique()
import statsmodels.api as sm

for date in dates:

  # Create the design matrix X
  crsp_temp=crsp[crsp['date']==date]
  X = crsp_temp[['market_equity_log','return_t-1', 'return_t-2_t-12', 'reported_book_to_market_log', 'retained_earnings_to_market_log', 'retained_earnings_to_market_neg', 'book_value_neg'  ]]
  X = sm.add_constant(X)

  # Create the target vector y
  y = crsp_temp['ret_adj']

  # Create the regression model without a constant
  model = sm.OLS(y, X)

  # Fit the model and print the summary
  results = model.fit()
  params = results.params
  adj_r2 = results.rsquared_adj

  #print(params)
  new_row = {'Date': date, 'ME': params[1]*100, 'R1': params[2]*100, 'R12': params[3]*100, 'BM':params[4]*100, 'RE':params[5]*100, 'I':params[6]*100, 'I2':params[7]*100,  'r2':adj_r2*100}
  df = df.append(new_row, ignore_index=True)
  print(date)

a.loc[a['regressor'] == 'log(me)', '(2)'] = "{:.2f} ({:.2f})".format(df['ME'].mean(), df['ME'].mean()/(df['ME'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R1', '(2)'] = "{:.2f} ({:.2f})".format(df['R1'].mean(), df['R1'].mean()/(df['R1'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'R12', '(2)'] = "{:.2f} ({:.2f})".format(df['R12'].mean(), df['R12'].mean()/(df['R12'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(Reported BE/ME)', '(2)'] = "{:.2f} ({:.2f})".format(df['BM'].mean(), df['BM'].mean()/(df['BM'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'log(re/me)', '(2)'] = "{:.2f} ({:.2f})".format(df['RE'].mean(), df['RE'].mean()/(df['RE'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'RE<0', '(2)'] = "{:.2f} ({:.2f})".format(df['I'].mean(), df['I'].mean()/(df['I'].std()/np.sqrt(len(df))))
a.loc[a['regressor'] == 'BE<0', '(2)'] = "{:.2f} ({:.2f})".format(df['I2'].mean(), df['I2'].mean()/(df['I2'].std()/np.sqrt(len(df))))

a.loc[a['regressor'] == 'Average ADJ R2', '(2)'] = df['r2'].mean()

In [ ]:
a.fillna('-', inplace=True)
a.set_index('regressor', inplace=True)
table2_c=a
table2_c

# Table 3

In [ ]:
crsp.sort_values(by=['permno', 'date'], inplace=True)

In [ ]:
crsp.set_index('date', inplace=True)

In [ ]:
crsp['ret_adj']

In [ ]:
for x in range(1, 13):
  crsp[f't+{x}']=crsp.groupby('permno')['ret_adj'].shift(-x)

for x in range(1, 13):
  #crsp[f't+{x}']/=100
  #crsp[f't+{x}']+=1
  '''
  #crsp['return_t+2_t+12'] = np.cumprod(crsp.loc[:, 't+1':'t+12'], axis=1).iloc[:, -1]

  # Subtract 1 and convert the result back to percentage returns
  #crsp['return_t+2_t+12'] = (crsp['return_t+2_t+12'] - 1)
  '''

In [ ]:
crsp.reset_index(inplace=True)
crsp['date']=pd.to_datetime(crsp['date'])

In [ ]:
#get breakpoints for book value

In [ ]:
crsp['date']=pd.to_datetime(crsp['date'])
bb=crsp[(crsp['date'].dt.month==6) & (crsp['exchange']=="NYSE")]
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT permno, reference_date, book_to_market as book_to_market
        FROM bb
        '''

bb=pd.read_sql(query, conn)

bb['quintile_1'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.2))
bb['quintile_2'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.4))
bb['quintile_3'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.6))
bb['quintile_4'] = bb.groupby('reference_date')['book_to_market'].transform(lambda x: x.quantile(0.8))


In [ ]:
crsp['month']=crsp['date'].dt.month

In [ ]:
crsp_june=crsp[crsp['month']==6]

In [ ]:
crsp_june

In [ ]:
bb.to_sql('bb', conn, if_exists='replace')
query=''' 
      SELECT DISTINCT reference_date, quintile_1, quintile_2, quintile_3, quintile_4
      FROM bb
          '''

bb=pd.read_sql(query, conn)
  

In [ ]:
bb

In [ ]:
crsp_june.to_sql('crsp', conn, if_exists='replace')
bb.to_sql('bb', conn, if_exists='replace')

query='''
        SELECT DISTINCT crsp.*, bb.quintile_1, bb.quintile_2, bb.quintile_3, bb.quintile_4
        FROM crsp
        LEFT JOIN bb
        ON crsp.reference_date=bb.reference_date
        '''

crsp_bb=pd.read_sql(query, conn)

In [ ]:
crsp=crsp_bb

In [ ]:
# define the quantile columns and the corresponding portfolio numbers
quantile_columns = ['quintile_1', 'quintile_2', 'quintile_3', 'quintile_4']
portfolio_numbers = [1, 2, 3, 4, 5]

# create a new column in crsp that assigns a security to a portfolio based on the quantiles
crsp['portfolio'] = np.where(crsp['book_to_market'] <= crsp[quantile_columns[0]], 
                             portfolio_numbers[0], 
                             np.where(crsp['book_to_market'] <= crsp[quantile_columns[1]],
                                      portfolio_numbers[1],
                                      np.where(crsp['book_to_market'] <= crsp[quantile_columns[2]],
                                               portfolio_numbers[2],
                                               np.where(crsp['book_to_market'] <= crsp[quantile_columns[3]],
                                                        portfolio_numbers[3],
                                                        portfolio_numbers[4]))))

In [ ]:
crsp['market_equity'].isnull().sum()

0

In [ ]:
crsp.sort_values(by='date')

In [ ]:
crsp['t+1']

0         0.030000
1         0.035714
2         0.025641
3        -0.035714
4         0.063830
            ...   
170572    0.250745
170573   -0.069816
170574   -0.007866
170575    0.106039
170576   -0.105473
Name: t+1, Length: 170577, dtype: float64

In [ ]:
crsp.dropna(subset=['return_t+2_t+12'], inplace=True)

In [ ]:
crsp = crsp.dropna(subset=['t+1', 't+2', 't+3', 't+4', 't+5', 't+6', 't+7', 't+8', 't+9', 't+10', 't+11', 't+12'])


In [ ]:
def weighted_average(x):
    weights = x['mkt_cap']
    return np.average(x.loc[:, 't+1':'t+12'], weights=weights, axis=0)

# Apply the function to the dataframe, grouping by date and portfolio
result = crsp.groupby(['date', 'portfolio']).apply(weighted_average)


In [ ]:
result.to_frame().reset_index()
portfolio_avg_df=result
portfolio_avg_df['weighted_avg'] = portfolio_avg_df[0].apply(lambda x: np.mean(x, axis=1))


AttributeError: ignored

In [ ]:
result.to_frame().reset_index()

df=result

In [ ]:
df=portfolio_avg_df.to_frame().reset_index()

In [ ]:
df['average_return'] = df[0].apply(lambda x: np.mean(x, axis=0))


In [ ]:
df

,date,portfolio,0,average_return
0,1965-06-30 00:00:00,1,"[0.017228825187163236, 0.02766880348214536, 0....",0.003355
1,1965-06-30 00:00:00,2,"[0.0014118599732376497, 0.010698300085444899, ...",-0.000789
2,1965-06-30 00:00:00,3,"[0.018701822490038985, 0.02374491504786335, 0....",0.000491
3,1965-06-30 00:00:00,4,"[0.02575269578226031, 0.04248005216169942, 0.0...",0.013094
4,1965-06-30 00:00:00,5,"[0.03297212588444094, 0.01895648496342636, 0.0...",0.008613
...,...,...,...,...
260,2017-06-30 00:00:00,1,"[0.4465955, 0.355691, 0.2535575, 0.151424, 0.2...",0.130475
261,2017-06-30 00:00:00,2,"[-0.013935999999999999, 0.044841, 0.044841, 0....",0.017457
262,2017-06-30 00:00:00,3,"[0.03729648030678315, 0.02912400225891614, 0.0...",0.018031
263,2017-06-30 00:00:00,4,"[-0.020954999999999998, 0.061538, 0.0778705, 0...",0.066943


In [ ]:
returns=df.groupby('portfolio')['average_return'].mean().reset_index()

In [ ]:
returns['average_return']*=100

In [ ]:
returns

,portfolio,average_return
0,1,1.060022
1,2,0.979117
2,3,0.969626
3,4,1.198721
4,5,1.299668


In [ ]:
crsp.columns

Index(['level_0', 'date', 'Unnamed: 0.1', 'Unnamed: 0', 'index', 'permno',
       'ret', 'shrout', 'altprc', 'exchcd', 'siccd', 'vol', 'mkt_cap',
       'exchange', 'ret_adj', 'reference_date', 'return_t-1', 'return_t-2',
       'return_t-3', 'return_t-4', 'return_t-5', 'return_t-6', 'return_t-7',
       'return_t-8', 'return_t-9', 'return_t-10', 'return_t-11', 'return_t-12',
       'gvkey', 'seq', 'ceq', 'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv',
       'pstkl', 'pstk', 'revt', 'cogs', 'xint', 'xsga', 'xrd', 'rect', 'invt',
       'xpp', 'drc', 'drlt', 'ap', 'xacc', 'cstk', 'caps', 'tstk', 're',
       'acominc', 'mkvalt', 'book_value', 'market_equity', 'cc',
       'book_to_market', 'reported_book_to_market',
       'contributed_capital_to_market', 'retained_earnings_to_market',
       'other_book_to_market', 're/b', 'cc/b', 'other/b', 'return_t-2_t-12',
       'contributed_capital_to_market_neg', 'retained_earnings_to_market_neg',
       'other_book_to_market_neg', 'market_equi

In [ ]:
portfolio_avg = portfolio_avg.to_frame().reset_index()


In [ ]:
portfolio_avg

,portfolio,date,0
0,1,1965-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
1,1,1966-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
2,1,1967-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
3,1,1968-06-28 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
4,1,1969-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
...,...,...,...
260,5,2013-06-28 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
261,5,2014-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
262,5,2015-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."
263,5,2016-06-30 00:00:00,"[0.004174673951960097, -0.003617936102806046, ..."


In [ ]:
portfolio_returns = portfolio_avg.groupby('portfolio')[0].mean().reset_index()


In [ ]:
portfolio_returns